In [33]:
import pandas as pd
import numpy as np
import matplotlib as plt

Import datasets

In [34]:
d13 = pd.read_stata('./data/2013', convert_categoricals=False)
d14 = pd.read_stata('./data/2014', convert_categoricals=False)
d15 = pd.read_stata('./data/2015', convert_categoricals=False)
d16 = pd.read_stata('./data/2016', convert_categoricals=False)
d17 = pd.read_stata('./data/2017', convert_categoricals=False)
d18 = pd.read_stata('./data/2018', convert_categoricals=False)
d19 = pd.read_stata('./data/2019', convert_categoricals=False)
d20 = pd.read_stata('./data/2020', convert_categoricals=False)
d21 = pd.read_stata('./data/2021', convert_categoricals=False)
d22 = pd.read_stata('./data/2022', convert_categoricals=False)

Recode pre-2017 LFSSTAT variables and rename EDUC90 to EDUC

In [84]:
pre = pd.concat([d13, d14, d15, d16])

def recode(LFSSTAT):
	if LFSSTAT == 4:
		return 3
	elif LFSSTAT == 5:
		return 3
	elif LFSSTAT == 6:
		return 4
	else:
		return LFSSTAT

pre.rename(columns={'EDUC90':'EDUC'}, inplace=True)

pre['LFSSTAT'] = pre['LFSSTAT'].apply(recode)

Append the rest of the data to the master

In [85]:
post = pd.concat([d17, d18, d19, d20, d21, d22])
master = pd.concat([pre, post])

Keep only prime age - 20 to 64 & Non-students

In [86]:
master = master[(master['AGE_12'] >= 2) & (master['AGE_12'] <= 10) & (master['SCHOOLN'] == 1)]

    QUESTION 1

        1a) Educational attainment

Generating new 'edu' variable and populating it

In [87]:
master['edu'] = np.nan

In [88]:
def recodeEDU(EDUC):
    if EDUC <= 1:
        return 1
    elif (EDUC == 2 or EDUC == 3):
        return 2
    elif EDUC == 4:
        return 3
    elif EDUC == 5:
        return 4
    elif EDUC == 6:
        return 5
    else:
        return EDUC

master['edu'] = master['EDUC'].apply(recodeEDU)

In [94]:
master['edu'].value_counts(normalize=True).sort_index()

1     69700
2    184382
3    260257
4    119043
5     51562
Name: edu, dtype: int64

    1b) Educational attainment over time